In [2]:
%matplotlib ipympl
from QDP import qdp
import os.path
import numpy as np
from scipy.stats import poisson
import matplotlib.pyplot as plt
from scipy import optimize
dp = '/home/ebert/jupyter-notebooks/FNode-data'
exclude = [2]

def gaussian(x, a, x0, s, b):
    return b + a*np.exp(-0.5*((x-x0)/s)**2)

In [3]:
exp_name = ''
if exp_name:
    exp_date = '_'.join(exp_name.split('_')[:3])
    experiment_file = os.path.join(exp_date, exp_name, 'results.hdf5')
else:
    experiment_file = ''
q = qdp.QDP(base_data_path=dp)
q.load_data_file(experiment_file)
print "ivars: {}".format(q.experiments[0]['variable_list'])
cs_cuts = [[170],[3]]
rb_cuts = [[110],[3]]
q.set_thresholds(cs_cuts, roi=0)
q.set_thresholds(rb_cuts, roi=1)
retention = q.apply_thresholds(exclude_rois=exclude)
cuts = q.get_thresholds()
roi_map = ['cs', 'rb']
fmt_map = ['bo', 'rx']
entries = np.product(retention['retention'].shape[:-1])
tbl_str = "ivar: {0:.4f}\tcloading: {1:.3f}\trloading: {2:.3f}\t\tcretention: {3:.3f} +- {4:.3f}\trretention: {5:.3f} +- {6:.3f}"
for i in range(entries):
    print(tbl_str.format(
        retention['ivar'].reshape((entries,-1))[i][0],
        retention['loading'].reshape((entries,-1))[i][0], retention['loading'].reshape((entries,-1))[i][1],
        retention['retention'].reshape((entries,-1))[i][0], retention['error'].reshape((entries,-1))[i][0],
        retention['retention'].reshape((entries,-1))[i][1], retention['error'].reshape((entries,-1))[i][1]
    ))
q.save_experiment_data()
xlab = q.experiments[0]['variable_desc'][q.experiments[0]['variable_list'][0]]['description']

# plotting
fig, ax = plt.subplots(nrows=2, figsize=(5,8))
plt.subplots_adjust(wspace=0.3)
for i, yvals in enumerate(['retention', 'loading']):
    for e in range(len(retention['ivar'])):
        for r in range(2):            
            if yvals=='retention':
                try:
                    ax[i].errorbar(retention['ivar'][e, :, r], retention[yvals][e, :, r],
                        yerr=retention['error'][e,:,r],
                        fmt=fmt_map[r]
                    )
                except Exception as ee:
                    print(ee)
                    ax[i].plot(retention['ivar'][e, :, 0], retention[yvals][e, :, r], fmt_map[r])
                # fitting
                xs = np.linspace(min(retention['ivar'][e, :, 0]), max(retention['ivar'][e, :, 0]),100)
                guess = [0.6, retention['ivar'][e, np.argmin(retention[yvals][e, :, r]), 0], 0.4, .0]
                try:
                    popt, pcov = optimize.curve_fit(
                        gaussian,
                        retention['ivar'][e, :, 0],
                        retention[yvals][e, :, r],
                        p0=guess,
                        sigma=retention['error'][e, :, r]
                    )
                except Exception as ee:
                    print ee
                else:
                    ax[i].plot(xs, gaussian(xs, *popt), fmt_map[r][0]+'--', alpha=0.6)
                    print(popt)
                    fit_data = "{}\na: {:.3f}\nx0: {:.3f}\ns: {:.3f}\nb: {:.3f}"
                    start = np.min(retention['ivar'][e, :, 0])
                    stop = np.max(retention['ivar'][e, :, 0])
                    dx = (stop-start)/10
                    ax[i].text(start+dx+r*5*dx, 0.5, fit_data.format(roi_map[r],*popt), fontsize=12)
            else:
                ax[i].plot(retention['ivar'][e, :, 0], retention[yvals][e, :, r], fmt_map[r])
                
        
    ax[i].set_ylim(0.8, 1)
    ax[i].set_xlabel(xlab)
    ax[i].set_ylabel(yvals)

fmts = ['pdf', 'png']
fn = os.path.join(dp, q.experiments[0]['source_path'], 'loading_scan.{}')
for fmt in fmts:
    fig.savefig(fn.format(fmt), dpi=200, format=fmt)

name 'fort_exp' is not defined
name 'rb_uwave_freq' is not defined
ivars: [u'Iy_read']
no loading observed
no loading observed
no loading observed
no loading observed
no loading observed
no loading observed
no loading observed
no loading observed
no loading observed
no loading observed
ivar: 0.1590	cloading: 0.257	rloading: 0.120		cretention: 0.934 +- 0.037	rretention: 0.642 +- 0.037
ivar: 0.1990	cloading: 0.234	rloading: 0.148		cretention: 0.910 +- 0.025	rretention: 0.851 +- 0.025
ivar: 0.2390	cloading: 0.203	rloading: 0.139		cretention: 0.926 +- 0.027	rretention: 0.820 +- 0.027
ivar: 0.2790	cloading: 0.308	rloading: 0.144		cretention: 0.958 +- 0.028	rretention: 0.806 +- 0.028
ivar: 0.3190	cloading: 0.235	rloading: 0.147		cretention: 0.915 +- 0.020	rretention: 0.905 +- 0.020
ivar: 0.3590	cloading: 0.228	rloading: 0.141		cretention: 0.934 +- 0.024	rretention: 0.872 +- 0.024
ivar: 0.3990	cloading: 0.284	rloading: 0.178		cretention: 0.838 +- 0.019	rretention: 0.893 +- 0.019
ivar: 0.4390	

FigureCanvasNbAgg()

Optimal parameters not found: Number of calls to function has reached maxfev = 1000.
[ 141.33402298    0.4520706     5.49225684 -140.4072197 ]


In [1]:
# plotting
fig, ax = plt.subplots(nrows=2, figsize=(5,8))
plt.subplots_adjust(wspace=0.3)
for i, yvals in enumerate(['retention', 'loading']):
    for e in range(len(retention['ivar'])):
        for r in range(2):            
            if yvals=='retention':
                try:
                    ax[i].errorbar(retention['ivar'][e, :, r], retention[yvals][e, :, r],
                        yerr=retention['error'][e,:,r],
                        fmt=fmt_map[r]
                    )
                except Exception as ee:
                    print(ee)
                    ax[i].plot(retention['ivar'][e, :, 0], retention[yvals][e, :, r], fmt_map[r])
                # fitting
                xs = np.linspace(min(retention['ivar'][e, :, 0]), max(retention['ivar'][e, :, 0]),100)
                guess = [0.6, retention['ivar'][e, np.argmin(retention[yvals][e, :, r]), 0], 0.2, .6]
                try:
                    popt, pcov = optimize.curve_fit(
                        gaussian,
                        retention['ivar'][e, :, 0],
                        retention[yvals][e, :, r],
                        p0=guess,
#                         sigma=retention['error'][e, :, r]
                    )
                except Exception as ee:
                    print ee
                else:
                    ax[i].plot(xs, gaussian(xs, *popt), fmt_map[r][0]+'--', alpha=0.6)
                    print(popt)
                    fit_data = "{}\na: {:.3f}\nx0: {:.3f}\ns: {:.3f}\nb: {:.3f}"
                    start = np.min(retention['ivar'][e, :, 0])
                    stop = np.max(retention['ivar'][e, :, 0])
                    dx = (stop-start)/10
                    ax[i].text(start+dx+r*6*dx, 0.1, fit_data.format(roi_map[r],*popt), fontsize=12)
            else:
                ax[i].plot(retention['ivar'][e, :, 0], retention[yvals][e, :, r], fmt_map[r])
                
        
    ax[i].set_ylim(0., 1.05)
    ax[i].set_xlabel(xlab)
    ax[i].set_ylabel(yvals)

fmts = ['pdf', 'png']
fn = os.path.join(dp, q.experiments[0]['source_path'], 'loading_scan.{}')
for fmt in fmts:
    fig.savefig(fn.format(fmt), dpi=200, format=fmt)

NameError: name 'plt' is not defined